In [43]:
# !pip install spacy

In [44]:
# !python -m spacy download en_core_web_sm

In [5]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
content = '''
The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
'''
doc = nlp(content)
print(doc)


The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.



In [28]:
s= ''
a = 'hello.'
b = 'world.'
l = [a,b]
l

['hello.', 'world.']

In [33]:
' '.join(l)

'hello. world.'

In [53]:
with open("maltext.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
content = ' '.join(lines)
doc = nlp(content)
print(doc)

The attacks associated with the new botnet attempted to exploit the CVE-2017-17215 zero-day vulnerability in the Huawei home router caused by the fact that the TR-064 technical report standard, which was designed for local network configuration, was exposed to WAN through port 37215 (UPnP – Universal Plug and Play).
 About 12 hours ago (2017-12-05 11:57 AM GMT+8), we noticed a new version of Satori (a mirai variant which we named Satori), starting to propagate very quickly on port 37215 and 52869.
 Two new exploits, which work on port 37215 and 52869 have been added, see below for more details.
 Due to the worm like behavior, we all should be on the lookout for the port 37215 and 52869 scan traffic.
 For those who don’t have the visibility, feel free to check out our free Scanmon system for port 37215 and 52869, or ISC port pages for 37215 and 52869.
 As can be seen from the following picture, the bot will scan port 37215 and 52869 randomly, determined by the remainder of a random inte

In [54]:
for sents in doc.sents:
    print(sents.text)

The attacks associated with the new botnet attempted to exploit the CVE-2017-17215 zero-day vulnerability in the Huawei home router caused by the fact that the TR-064 technical report standard, which was designed for local network configuration, was exposed to WAN through port 37215 (UPnP – Universal Plug and Play).
 
About 12 hours ago (2017-12-05 11:57 AM GMT+8), we noticed a new version of Satori (a mirai variant which we named Satori), starting to propagate very quickly on port 37215 and 52869.
 
Two new exploits, which work on port 37215 and 52869 have been added, see below for more details.
 
Due to the worm like behavior, we all should be on the lookout for the port 37215 and 52869 scan traffic.
 
For those who don’t have the visibility, feel free to check out our free Scanmon system for port 37215 and 52869, or ISC port pages for 37215 and 52869.
 
As can be seen from the following picture, the bot will scan port 37215 and 52869
randomly, determined by the remainder of a random

In [55]:
candidate_pos = ['NOUN', 'PROPN', 'VERB']
sentences = []

for sent in doc.sents:
    selected_words = []
    for token in sent:
        # print(token.pos_, token.is_stop)
        if token.pos_ in candidate_pos and token.is_stop is False:
            selected_words.append(token)
    sentences.append(selected_words)

print(sentences, len(sentences))

[[attacks, associated, new, botnet, attempted, exploit, CVE-2017, day, vulnerability, Huawei, home, router, caused, fact, TR-064, report, standard, designed, network, configuration, exposed, WAN, port, UPnP, Universal, Plug, Play], [hours, GMT+8, noticed, version, Satori, mirai, named, Satori, starting, propagate, port], [exploits, work, port, added, details], [worm, behavior, lookout, port, traffic], [visibility, feel, check, Scanmon, system, port, ISC, port, pages], [seen, picture, bot, scan, port], [determined, remainder, integer, mod], [scanning, Satori, utilize, exploits, port], [port, disclosed, team, tracking, days, got, insight, discuss, now.(stay, tuned, update], [example, hours, seen, IPs, scanning, port, IPs, scanning, port], [analyzed, Mirai, variant, called, Miori, spread, Remote, Code, Execution, RCE, vulnerability, PHP, framework], [arrival, method, IoT, botnet, uses, said, exploit, affects, versions], [analysis, revealed, cybercriminals, Miori, ThinkPHP, RCE, machines, 

In [91]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm

nlp = en_core_web_sm.load()

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        allans = []
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
#             print(key + ' - ' + str(value))
            ans = key + ' - ' + str(value)
            allans.append(ans)
            if i > number:
                break
        return allans
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN', 'NUM'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [92]:
tr4w = TextRank4Keyword()
tr4w.analyze(content, candidate_pos = ['NOUN', 'PROPN', 'NUM'], window_size=4, lower=False)
tr4w.get_keywords(10)

['port - 7.92753676044077',
 'file - 5.12354157537708',
 'Emotet - 4.390962629052478',
 '80 - 4.1796382207502925',
 '37215 - 3.102278261398497',
 '.doc - 2.6395333990358423',
 'email - 2.567319550880556',
 'PDF - 2.5312661647572186',
 'exploit - 2.509049025713741',
 'code - 2.4569210720766845',
 '8080 - 2.4461717955760798',
 'malware - 2.402024034351282']

In [93]:
a = tr4w.get_keywords(5)
a

['port - 7.92753676044077',
 'file - 5.12354157537708',
 'Emotet - 4.390962629052478',
 '80 - 4.1796382207502925',
 '37215 - 3.102278261398497',
 '.doc - 2.6395333990358423',
 'email - 2.567319550880556']

In [85]:
a

['port - 9.19284380563609',
 'file - 5.133954780338477',
 'Emotet - 4.4817964246789',
 '.doc - 2.665007041755904',
 'email - 2.5764201116060534',
 'PDF - 2.5431762319014917',
 'code - 2.4450875393429827']

In [101]:
tr4w = TextRank4Keyword()
tr4w.analyze("Emotet is most commonly spread via malicious emails containing Microsoft Office attachments, usually Microsoft Word (.doc, .docx) documents.", candidate_pos = ['NOUN', 'PROPN', 'NUM'], window_size=4, lower=False)
b = tr4w.get_keywords(0)
b

['Microsoft - 1.7967569444444442', 'attachments - 1.1626333333333334']

In [118]:
mal = nlp("About 12 hours ago (2017-12-05 11:57 AM GMT+8), we noticed a new version of Satori (a mirai variant which we named Satori), starting to propagate very quickly on port 37215 and 52869.")

In [119]:
for word in mal:
    print(word.text, word.pos_)

About ADV
12 NUM
hours NOUN
ago ADV
( PUNCT
2017 NUM
- PUNCT
12 NUM
- SYM
05 NUM
11:57 NUM
AM PROPN
GMT+8 NOUN
) PUNCT
, PUNCT
we PRON
noticed VERB
a DET
new ADJ
version NOUN
of ADP
Satori PROPN
( PUNCT
a DET
mirai NOUN
variant ADJ
which DET
we PRON
named VERB
Satori PROPN
) PUNCT
, PUNCT
starting VERB
to PART
propagate VERB
very ADV
quickly ADV
on ADP
port NOUN
37215 NUM
and CCONJ
52869 NUM
. PUNCT


In [125]:
spacy.explain('ORG')
spacy.explain("PERSON")

'People, including fictional'

In [126]:
from spacy import displacy

displacy.render(mal, style='ent', jupyter=True)